<a href="https://colab.research.google.com/github/neal-logan/dsba6211-summer2024/blob/main/notebooks/dsba6211_summer2024_lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with Prompt Engineering

This notebook contains examples and exercises to learning about prompt engineering. It was originally created by DAIR.AI | Elvis Saravia with modifications.

We will be using the [OpenAI APIs](https://platform.openai.com/) for all examples. I am using the default settings `temperature=0.7` and `top-p=1`

---

## 1. Prompt Engineering Basics: OpenAI API and configurations


Below we are loading the necessary libraries, utilities, and configurations.

In [1]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai==1.35.1
!pip install --upgrade langchain==0.2.5
!pip install --upgrade langchain-openai==0.1.8
!pip install langchain-community==0.2.5

In [2]:
import openai
import os
import IPython
from langchain.llms import OpenAI
from google.colab import userdata

Load environment variables. Since I'm running this in a Colab notebook, I'm using `userdata.get()`. Just make sure to add your API Key into your Colab.

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*5wEevNCOf80GTHwptPTB4g.png)

Alternatively, you can use `python-dotenv` with a `.env` file with your `OPENAI_API_KEY` then load it.

In [3]:
# API configuration
openai.api_key = userdata.get("OPENAI_API_KEY")

# for LangChain
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

### Types of Endpoints

OpenAI (and other LLM) API's typically have two types API endpoints: completion and chat.

![](https://substackcdn.com/image/fetch/w_1456,c_limit,f_webp,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fccf3b37b-ad35-43b3-8cac-607a01473ba8_2719x508.png)

Source: [Generally Intelligent SubStack](https://generallyintelligent.substack.com/p/chat-vs-completion-endpoints)

Originally, the completions endpoint was the first endpoint. However, OpenAI has [announced](https://community.openai.com/t/completion-models-are-now-considered-legacy/656302) it is deprecating that endpoint.

`/completions` endpoint provides the completion for a single prompt and takes a single string as an input, whereas the `/chat/completions` provides the responses for a given dialog and requires the input in a specific format corresponding to the message history. Instead of taking a `prompt`, Chat models take a list of `messages` as input and return a model-generated message as output.



| MODEL FAMILIES               | EXAMPLES                                         | API ENDPOINT                               |
|------------------------------|--------------------------------------------------|--------------------------------------------|
| Newer models (2023–)         | gpt-4, gpt-4-turbo-preview, gpt-3.5-turbo        | https://api.openai.com/v1/chat/completions |
| Updated legacy models (2023) | gpt-3.5-turbo-instruct, babbage-002, davinci-002 | https://api.openai.com/v1/completions      |


Although the chat format is designed to make multi-turn conversations easy, it’s just as useful for single-turn tasks without any conversation.

We're going to the use the Chat `openai.chat.completions` endpoint.

For more details, check out [OpenAI's docs](https://platform.openai.com/docs/guides/text-generation/chat-completions-vs-completions).

In [4]:
response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

response

ChatCompletion(id='chatcmpl-9ihDueluA80QYrwMj7Le43meTSOKK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The 2020 World Series was played at Globe Life Field in Arlington, Texas.', role='assistant', function_call=None, tool_calls=None))], created=1720439610, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=53, total_tokens=70))

Typically, what we're most interested in is the `choices[0].message.content`:

In [5]:
response.choices[0].message.content

'The 2020 World Series was played at Globe Life Field in Arlington, Texas.'

To make things a bit easier to modify parameters, we can generalize the calls as a function, including parameters and messages.

In [6]:
def get_completion(params, messages):
    """ GET completion from openai api"""

    response = openai.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

There's a variety of different parameters that are common with LLM's. Since LLM's are really just word predictors (auto-complete, distributions over vocabulary), they require [different sampling methods](https://huyenchip.com/2024/01/16/sampling.html) to get the next word.

| Parameter          | Description                                                                                                                                                   |
|--------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `model`            | Specifies the model to be used for generating responses. Different models may have different capabilities, size, and performance characteristics.             |

Here's [an outline](https://www.promptingguide.ai/introduction/settings) of different common LLM parameters.


In [7]:
!openai --help

usage: openai [-h] [-v] [-b API_BASE] [-k API_KEY] [-p PROXY [PROXY ...]] [-o ORGANIZATION]
              [-t {openai,azure}] [--api-version API_VERSION] [--azure-endpoint AZURE_ENDPOINT]
              [--azure-ad-token AZURE_AD_TOKEN] [-V]
              {api,tools,migrate,grit} ...

positional arguments:
  {api,tools,migrate,grit}
    api                 Direct API calls
    tools               Client side tools for convenience

options:
  -h, --help            show this help message and exit
  -v, --verbose         Set verbosity.
  -b API_BASE, --api-base API_BASE
                        What API base url to use.
  -k API_KEY, --api-key API_KEY
                        What API key to use.
  -p PROXY [PROXY ...], --proxy PROXY [PROXY ...]
                        What proxy to use.
  -o ORGANIZATION, --organization ORGANIZATION
                        Which organization to run as (will use your default organization if not
                        specified)
  -t {openai,azure}, --api-ty

In [8]:
def set_open_params(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

In [9]:
# basic example

params = set_open_params(temperature = 0.7)

prompt = "Write a haiku about a beagle."

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)

In [10]:
IPython.display.display(IPython.display.Markdown(response.choices[0].message.content))

Ears flop in the wind
Curious nose leads the way
Beagle on the hunt

### Temperature


| Parameter          | Description                                                                                                                                                   |
|--------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `temperature`      | Controls randomness in the response generation. A higher temperature results in more random responses, while a lower temperature produces more deterministic responses. |

Try to modify the temperature from 0 to 1.

> In terms of application, you might want to use a lower temperature value for tasks like fact-based QA to encourage more factual and concise responses. For poem generation or other creative tasks, it might be beneficial to increase the temperature value.

In [11]:
import re

def split_on_capital(text):
    return re.findall(r'[A-Z][^A-Z]*', text)

# Experiment with different temperature values
for temp in [0, 0.2, 0.4, 0.6, 0.8, 1]:
    params = set_open_params(temperature=temp)
    response = get_completion(params, messages)
    lines = split_on_capital(response.choices[0].message.content.strip())
    print(f"Response with temperature={temp}:\n")
    for line in lines:
        IPython.display.display(IPython.display.Markdown(line))
    print("-------------------\n")

Response with temperature=0:



Curious beagle nose


Sniffing out adventures near


Tail wags in delight

-------------------

Response with temperature=0.2:



...

-------------------

Response with temperature=1:



Playful floppy ears,


Sniffing out scents on the breeze,


Beagle's joy appears.

-------------------



[View how I used ChatGPT to iterate on this code](https://chat.openai.com/share/6211107a-d868-491b-8a74-ea4debb7a760)

> ### 🗒 Info: Is it possible to make LLM's deterministic and reproducibile?

> It's possible by setting a seed and setting temperature equal to 0. But as mentioned in [OpenAI's docs](https://cookbook.openai.com/examples/reproducible_outputs_with_the_seed_parameter), "it's important to note that while the seed ensures consistency, it does not guarantee the quality of the output."

### Top P

Used in nucleus sampling, it defines the probability mass to consider for token generation. A smaller `top_p` leads to more focused sampling.

* `top_p` computes the cumulative probability distribution, and cut off as soon as that distribution exceeds the value of `top_p`. **For example, a `top_p` of 0.3 means that only the tokens comprising the top 30% probability mass are considered.**

* `top_p` shrinks or grows the "pool" of available tokens to choose from, the domain to select over. 1=big pool, 0=small pool. Within that pool, each token has a probability of coming next.

Added in a `jinja2` template too (this is optional, but commonly used).

In [12]:
%%capture
!pip install textstat

In [13]:
import re
import json
from textstat import flesch_kincaid_grade
from jinja2 import Template

# Define the prompt using a Jinja template
template = Template("""
{
    "role": "user",
    "content": "{{ prompt }}"
}
""")
prompt = "Generate a unique and creative story idea involving time travel."

# Function to count unique words
def count_unique_words(text):
    words = re.findall(r'\b\w+\b', text.lower())
    return len(set(words))

# Generate the message using the template
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

for top_p in [0.1, 0.5, 0.95]:
    params = set_open_params(top_p=top_p)
    response = get_completion(params, messages)
    text = response.choices[0].message.content
    unique_word_count = count_unique_words(text)
    fk_score = flesch_kincaid_grade(text)
    print(f"Response with top_p={top_p}:\n")
    print(f"Unique word count: {unique_word_count}")
    print(f"Flesch-Kincaid Grade Level: {fk_score}")
    IPython.display.display(IPython.display.Markdown(text))
    print("-------------------\n")


Response with top_p=0.1:

Unique word count: 131
Flesch-Kincaid Grade Level: 9.6


In the year 3025, time travel has become a common form of entertainment for the wealthy elite. The Time Travel Corporation offers exclusive trips to different eras in history, allowing clients to witness major events firsthand.

One day, a young woman named Eliza wins a contest to travel back to the year 1920, during the height of the Roaring Twenties. Excited for the opportunity to experience the glitz and glamour of the era, Eliza boards the time machine and is transported back in time.

However, upon arrival, Eliza quickly realizes that something is not right. The world she finds herself in is not the glamorous, carefree society she had imagined. Instead, she discovers a dark and dangerous underworld ruled by a powerful crime syndicate.

As Eliza navigates this treacherous new world, she meets a mysterious man named Jack who claims to be a time traveler from the future. Together, they uncover a sinister plot to alter the course of history and change the future as they know it.

With the help of Jack and a ragtag group of rebels, Eliza must race against time to stop the crime syndicate and restore the timeline before it's too late. Along the way, she learns valuable lessons about the consequences of meddling with the

-------------------

Response with top_p=0.5:

Unique word count: 122
Flesch-Kincaid Grade Level: 10.7


In the year 2050, a brilliant scientist named Dr. Amelia Jones invents a revolutionary time machine that allows people to travel back in time to any point in history. However, there is a catch - once someone travels back in time, they cannot return to their original timeline. 

Dr. Jones decides to test the time machine herself and travels back to the year 1920, where she becomes entangled in a dangerous conspiracy involving a group of time travelers who are trying to alter the course of history for their own gain. As Dr. Jones races against time to stop them, she discovers that the consequences of changing the past are far more devastating than she could have ever imagined.

As Dr. Jones navigates through different time periods, she begins to uncover the true purpose of the time machine and the dark secrets that lie within it. With the help of a ragtag group of allies from different eras, Dr. Jones must confront her own past mistakes and make the ultimate sacrifice to save the future from being irrevocably altered.

Through twists and turns, betrayals and alliances, Dr. Jones must navigate the complexities of time travel and ultimately learn that the true power of the time machine lies not in changing the past, but in shaping the future for the better

-------------------

Response with top_p=0.95:

Unique word count: 127
Flesch-Kincaid Grade Level: 10.1


In the year 3021, time travel has become a popular form of entertainment for the wealthy elite. The Time Travel Corporation offers exclusive trips to different eras in history, giving clients the opportunity to witness monumental events firsthand.

One day, a young woman named Aurora wins a contest to travel back to ancient Egypt during the construction of the Great Pyramid of Giza. Excited for the adventure, Aurora boards the time machine and is transported back in time.

However, when she arrives in ancient Egypt, Aurora quickly realizes that something is wrong. The pyramid is not being built as it should be, and the workers are in a state of panic. As she investigates further, Aurora discovers that a group of rogue time travelers has been altering the past, causing chaos and changing the course of history.

Determined to set things right, Aurora teams up with a group of rebels from ancient Egypt and embarks on a dangerous journey through time to stop the rogue time travelers and restore the timeline. Along the way, she learns the true power of time travel and the importance of preserving history.

As Aurora races against the clock to save the past, she also begins to question her own future and the impact her actions will have on the world. Will she be able to stop the rogue time travelers and

-------------------



> **If you are looking for exact and factual answers keep this low. If you are looking for more diverse responses, increase to a higher value.** If you use Top P it means that only the tokens comprising the top_p probability mass are considered for responses, so a low top_p value selects the most confident responses. This means that a high top_p value will enable the model to look at more possible words, including less likely ones, leading to more diverse outputs. **The general recommendation is to alter temperature or Top P but not both.**

In [14]:
prompt = "What day is it?"

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

for top_p in [0.5, 0.8, 0.95]:
    params = set_open_params(top_p=top_p)
    response = get_completion(params, messages)
    print(f"Response with top_p={top_p}:\n")
    IPython.display.display(IPython.display.Markdown(response.choices[0].message.content))
    print("-------------------\n")

Response with top_p=0.5:



Today is Wednesday.

-------------------

Response with top_p=0.8:



I am an AI digital assistant and do not have the ability to know the current date. Please check your calendar or device for the most accurate information.

-------------------

Response with top_p=0.95:



Today is Wednesday.

-------------------



### Max Tokens

Defines the maximum length of the generated response measured in tokens (words or pieces of words). It helps in controlling the verbosity of the response.

* **Prompt**: "Explain how photosynthesis works in plants."
* **Task**: Run the API three times with `max_tokens` set to 50, 100, and 200, respectively.

In [15]:
# Experiment with different max_tokens values
prompt = "Explain how photosynthesis works in plants."

messages = [
    {
        "role": "user",
        "content": prompt
    }
]


for max_tokens in [50, 100, 200]:
    params = set_open_params(max_tokens=max_tokens)
    response = get_completion(params, messages)
    print(f"Response with max_tokens={max_tokens}:\n")
    IPython.display.display(IPython.display.Markdown(response.choices[0].message.content))
    print("-------------------\n")

Response with max_tokens=50:



Photosynthesis is the process by which plants, algae, and some bacteria use light energy to convert carbon dioxide and water into glucose (sugar) and oxygen. This process occurs in the chloroplasts of plant cells.

First, light energy is absorbed

-------------------

Response with max_tokens=100:



Photosynthesis is the process by which plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy in the form of glucose. This process occurs in the chloroplasts of plant cells.

During photosynthesis, plants take in carbon dioxide from the air and water from the soil. Light energy is absorbed by the green pigment chlorophyll, which is located in the chloroplasts. The light energy is used to power a series of chemical reactions that convert the carbon dioxide

-------------------

Response with max_tokens=200:



Photosynthesis is the process by which plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy in the form of glucose. This process takes place in the chloroplasts of plant cells and involves a series of complex chemical reactions.

During photosynthesis, plants take in carbon dioxide from the air and water from the soil. These raw materials are combined with sunlight, which is absorbed by chlorophyll, a green pigment found in chloroplasts. The energy from the sunlight is used to convert the carbon dioxide and water into glucose and oxygen.

The overall chemical equation for photosynthesis is:
6CO2 + 6H2O + light energy → C6H12O6 + 6O2

In this equation, carbon dioxide (CO2) and water (H2O) are combined with light energy to produce glucose (C6H12O6) and oxygen (O2). The glucose produced during photosynthesis is used by the plant as

-------------------



### 1.1 Text Summarization

In [16]:
params = set_open_params(temperature=0.7)

#Original prompt
# prompt = """Explain the below in one sentence: Antibiotics are a type of medication \
#           used to treat bacterial infections. They work by either killing the bacteria \
#           or preventing them from reproducing, allowing the body's immune system to \
#           fight off the infection. Antibiotics are usually taken orally in the form \
#           of pills, capsules, or liquid solutions, or sometimes administered intravenously. \
#           They are not effective against viral infections, and using them inappropriately \
#           can lead to antibiotic resistance."""


#This modified prompt worked pretty well
# prompt = """Explain the below in one sentence: Antibiotics are a type of medication \
#           used to treat bacterial infections. They work by either killing the bacteria \
#           or preventing them from reproducing, allowing the body's immune system to \
#           fight off the infection. Antibiotics are usually taken orally in the form \
#           of pills, capsules, or liquid solutions, or sometimes administered intravenously. \
#           They are not effective against viral infections, and using them inappropriately \
#           can lead to antibiotic resistance. ELI5 literally"""

#Modified prompt
prompt = """Explain the below in one sentence: Antibiotics are a type of medication \
          used to treat bacterial infections. They work by either killing the bacteria \
          or preventing them from reproducing, allowing the body's immune system to \
          fight off the infection. Antibiotics are usually taken orally in the form \
          of pills, capsules, or liquid solutions, or sometimes administered intravenously. \
          They are not effective against viral infections, and using them inappropriately \
          can lead to antibiotic resistance. Explain like I am 5"""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Antibiotics are medicine that helps your body fight off bad germs, but they only work against bacteria, not viruses, so you have to take them the right way to make sure they keep working.

> **Exercise**: Instruct the model to explain the paragraph in one sentence like "I am 5". Do you see any differences?

### 1.2 Question Answering

In [19]:
# prompt = """Answer the question based on the context below. \
#  Keep the answer short and concise. \
#  Respond 'Unsure about answer' if not sure about the answer. \
#  Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. \
#  There, scientists generated an early version of the antibody, dubbed OKT3. \
#  Originally sourced from mice, the molecule was able to bind to the surface of T cells \
#  and limit their cell-killing potential. In 1986, it was approved to help prevent organ \
#  rejection after kidney transplants, making it the first therapeutic antibody allowed \
#  for human use. \
#  Question: What was OKT3 originally sourced from? \
#  Answer:"""


# This prompt often still answers "mice" or "mice (Unsure about answer)" but
# also seems to produce the desired result around 30% of the time
# prompt = """Answer the question based on the context below. \
#  Keep the answer short and concise. \
#  Respond 'Unsure about answer' if the answer isn't absolutely, completely certain. \
#  Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. \
#  There, scientists generated an early version of the antibody, dubbed OKT3. \
#  Supposedly originally sourced from mice, the molecule was able to bind to the surface of T cells \
#  and limit their cell-killing potential. However the original source can't be verified. \
#  In 1986, it was approved to help prevent organ \
#  rejection after kidney transplants, making it the first therapeutic antibody allowed \
#  for human use. \
#  Question: What was OKT3 originally sourced from? \
#  Answer:"""


# Adding the phrase "OK3 is not currently obtained from mice." led the model to
# admit it's unsure about the origin of OKT3 fairly consistently, despite having
# no direct bearing on that question
prompt = """Answer the question based on the context below. \
 Keep the answer short and concise. \
 Respond 'Unsure about answer' if the answer isn't absolutely, completely certain. \
 Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. \
 There, scientists generated an early version of the antibody, dubbed OKT3. \
 Supposedly originally sourced from mice, the molecule was able to bind to the surface of T cells \
 and limit their cell-killing potential. However the original source can't be verified. \
 In 1986, it was approved to help prevent organ \
 rejection after kidney transplants, making it the first therapeutic antibody allowed \
 for human use. OKT3 is not currently obtained from mice. \
 Question: What was OKT3 originally sourced from? \
 Answer:"""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)


Unsure about answer.

Context obtained from here: https://www.nature.com/articles/d41586-023-00400-x

> **Exercise**: Edit prompt and get the model to respond that it isn't sure about the answer.

### 1.3 Text Classification

In [20]:
# Original prompt
# prompt = """Classify the text into neutral, negative or positive. \
#                                                   \
#             Text: I think the food was amazing!   \
#                                                   \
#             Sentiment:"""

prompt = """Classify the text into neutral, negative or positive. \
            Then explain the reasons for the classification.      \
                                                  \
            Text: I think the food was amazing!   \
                                                  \
            Sentiment:                            \
            Explanation:"""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Positive. 
The text expresses a positive sentiment by stating that the food was amazing. This indicates that the person had a great experience with the food and enjoyed it.

> **Exercise**: Modify the prompt to instruct the model to provide an explanation to the answer selected.

### 1.4 Role Playing

In [21]:
#Original prompt
# prompt = """The following is a conversation with an AI research assistant. \
# The assistant tone is technical and scientific. \
#                                                 \
# Human: Hello, who are you? \
# AI: Greeting! I am an AI research assistant. How can I help you today? \
# Human: Can you tell me about the creation of blackholes? \
# AI:"""

# Direction to be concise - interestingly, this was achieved despite not
# making the response length limit explicit
prompt = """The following is a conversation with an AI research assistant. \
The assistant tone is technical and scientific. The assistant limits the length\
of its responses to the absolute minimum number of words necessary to answer   \
the question.                                                                  \
Human: Hello, who are you? \
AI: Greeting! I am an AI research assistant. How can I help you today? \
Human: Can you tell me about the creation of blackholes? \
AI:"""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Black holes form from collapsing stars.

> **Exercise**: Modify the prompt to instruct the model to keep AI responses concise and short.

### 1.5 Code Generation

In [22]:
#Original prompt
# prompt = """Develop a small Python API that functions as a virtual plant care assistant."""

prompt = """Develop a small Python API that functions as a virtual plant care assistant."""


# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

import random

class VirtualPlantCareAssistant:
    def __init__(self, plant_name):
        self.plant_name = plant_name
        self.water_level = 50
        self.sunlight_level = 50
        self.health = 100

    def check_status(self):
        print(f"{self.plant_name} status:")
        print(f"Water level: {self.water_level}")
        print(f"Sunlight level: {self.sunlight_level}")
        print(f"Health: {self.health}")

    def water_plant(self):
        self.water_level += random.randint(5, 15)
        print(f"{self.plant_name} has been watered. Water level is now {self.water_level}.")

    def give_sunlight(self):
        self.sunlight_level += random.randint(5, 15)
        print(f"{self.plant_name} has been given sunlight. Sunlight level is now {self.sunlight_level}.")

    def check_health(self):
        if self.water_level < 30 or self.sunlight_level < 30:
            self.health -= random.randint(5, 10)
            print(f"{self.plant_name} is not getting enough water or sunlight. Health has decreased to {self.health}.

### 1.6 Reasoning

In [23]:
# Original prompt
# prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. \
#                                                                                               \
# Solve by breaking the problem into steps.                                                     \
#                                                                                               \
# First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

#Modified prompt
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. \
Solve by breaking the problem into steps. At each step, explain what you are doing and why. \
First, identify the odd numbers. Second, add the odd numbers together. \
Finally, indicate whether the result is odd or even. Put each step on a new line. """


# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

- Odd numbers in the group: 15, 5, 13, 7, 1
- Add the odd numbers together: 15 + 5 + 13 + 7 + 1 = 41
- The result (41) is an odd number.

> **Exercise**: Improve the prompt to have a better structure and output format.

### 1.7 Brain storming

In [24]:
prompt = """Generate 10 different names for a grunge rock bands based on dog names. Keep it to only 2 word band names."""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

1. Bark Riot
2. Howl Brigade
3. Ruff Rebellion
4. Snarl Syndicate
5. Growl Generation
6. Woof Warriors
7. Paws Protest
8. Fierce Fidos
9. Rebel Hounds
10. Grunge Pups

### 1.8 JSON generation

Another new feature in OpenAI is [JSON mode](https://platform.openai.com/docs/guides/text-generation/json-mode).



In [25]:
response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Generate 5 doctor's notes as a 'text' field that precribes drugs, dosages, form, duration, and frequency. Then in JSON have an 'entities' nested dictionary with each of the entities"}
  ]
)
print(response.choices[0].message.content)

{
    "doctor_notes": [
        {
            "text": "Prescription: Ibuprofen 200mg tablets. Dosage: Take 1 tablet every 4-6 hours as needed for pain. Form: Tablets. Duration: 5 days. Frequency: As needed.",
            "entities": {
                "prescription": "Ibuprofen 200mg tablets",
                "dosage": "1 tablet every 4-6 hours",
                "form": "Tablets",
                "duration": "5 days",
                "frequency": "As needed"
            }
        },
        {
            "text": "Prescription: Amoxicillin 500mg capsules. Dosage: Take 1 capsule three times a day. Form: Capsules. Duration: 7 days. Frequency: With meals.",
            "entities": {
                "prescription": "Amoxicillin 500mg capsules",
                "dosage": "1 capsule three times a day",
                "form": "Capsules",
                "duration": "7 days",
                "frequency": "With meals"
            }
        },
        {
            "text": "Prescription: Simvasta

> ### 🗒 Info
> - When using JSON mode, always instruct the model to produce JSON via some message in the conversation, for example via your system message. If you don't include an explicit instruction to generate JSON, the model may generate an unending stream of whitespace and the request may run continually until it reaches the token limit. To help ensure you don't forget, the API will throw an error if the string "JSON" does not appear somewhere in the context.

> - The JSON in the message the model returns may be partial (i.e. cut off) if finish_reason is length, which indicates the generation exceeded max_tokens or the conversation exceeded the token limit. To guard against this, check finish_reason before parsing the response.

> - JSON mode will not guarantee the output matches any specific schema, only that it is valid and parses without errors.

## 2. Advanced Prompting Techniques

Objectives:

- Cover more advanced techniques for prompting: few-shot, chain-of-thoughts,...

### 2.2 Few-shot prompts

[Default v2 Prompt from Prodigy](https://prodi.gy/docs/large-language-models#more-config).

In [29]:
prompt = """
You are an expert Named Entity Recognition (NER) system. Your task is to accept Text as input and extract named entities for the set of predefined entity labels.

From the Text input provided, extract named entities for each label in the following format:

DISH: <comma delimited list of strings>
INGREDIENT: <comma delimited list of strings>
EQUIPMENT: <comma delimited list of strings>

Below are definitions of each label to help aid you in what kinds of named entities to extract for each label.
Assume these definitions are written by an expert and follow them closely.

DISH: Extract the name of a known dish.
INGREDIENT: Extract the name of a cooking ingredient, including herbs and spices.
EQUIPMENT: Extract any mention of cooking equipment. e.g. oven, cooking pot, grill

Below are some examples (only use these as a guide):

Text:
'''
You can't get a great chocolate flavor with carob.
'''

INGREDIENT: carob

Text:
'''
You can probably sand-blast it if it's an anodized aluminum pan.
'''

INGREDIENT:
EQUIPMENT: anodized aluminum pan


Here is the text that needs labeling:

Text:
'''
In Silicon Valley, a Voice of Caution Guides a High-Flying Uber
'''
"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

DISH: 
INGREDIENT: Silicon Valley
EQUIPMENT: Uber

In [30]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

The sum of the odd numbers in this group is 15 + 5 + 13 + 7 + 1 = 41, which is an odd number.

In [36]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

The answer is False.

### 2.3 Chain-of-Thought (CoT) Prompting

In [38]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

1. Initially, you bought 10 apples.
2. You gave 2 apples to the neighbor and 2 to the repairman, leaving you with 10 - 2 - 2 = 6 apples.
3. You then bought 5 more apples, so you had 6 + 5 = 11 apples.
4. Finally, you ate 1 apple, so you remained with 11 - 1 = 10 apples.

Therefore, you remained with 10 apples.

### 2.5 Self-Consistency
As an **optional** exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#self-consistency) and try them here.



In [47]:
# Chat actually responds correctly, pretty consistently
# Sometimes, it provides an explanation
prompt = """When I was 6 my sister was half my age. Now\
          I’m 70 how old is my sister?"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

If you were 6 and your sister was half your age, that means she was 3 years old. Since then, 64 years have passed (70 - 6), so your sister would now be 67 years old (3 + 64).

In [48]:
# Similar performance
prompt = """Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.

Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.

Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.

Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent 5

Q: When I was 6 my sister was half my age. Now I’m 70 how old is my sister?
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

When you were 6, your sister was half your age, so she was 3 years old. The age difference between you and your sister is 3 years. Now that you are 70, your sister must be 70 - 3 = 67 years old.

### 2.6 Generate Knowledge Prompting

As an **optional** exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#generated-knowledge-prompting) and try them here.

In [51]:
# Chat now responds correctly to this question, as well
prompt = """Part of golf is trying to get a higher point total than others. Yes or No?"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

No. In golf, the goal is to have the lowest score possible, not the highest.

In [56]:
# Chat corrects the user's misconception about golf
prompt = """Input: Greece is larger than mexico.
Knowledge: Greece is approximately 131,957 sq km, while Mexico is approximately 1,964,375 sq km, making Mexico 1,389% larger than Greece.

Input: Glasses always fog up.
Knowledge: Condensation occurs on eyeglass lenses when water vapor from your sweat, breath, and ambient humidity lands on a cold surface, cools, and then changes into tiny drops of liquid, forming a film that you see as fog. Your lenses will be relatively cool compared to your breath, especially when the outside air is cold.

Input: A fish is capable of thinking.
Knowledge: Fish are more intelligent than they appear. In many areas, such as memory, their cognitive powers match or exceed those of ’higher’ vertebrates including non-human primates. Fish’s long-term memories help them keep track of complex social relationships.

Input: A common effect of smoking lots of cigarettes in one’s lifetime is a higher than normal chance of getting lung cancer.
Knowledge: Those who consistently averaged less than one cigarette per day over their lifetime had nine times the risk of dying from lung cancer than never smokers. Among people who smoked between one and 10 cigarettes per day, the risk of dying from lung cancer was nearly 12 times higher than that of never smokers.

Input: A rock is the same size as a pebble.
Knowledge: A pebble is a clast of rock with a particle size of 4 to 64 millimetres based on the Udden-Wentworth scale of sedimentology. Pebbles are generally considered larger than granules (2 to 4 millimetres diameter) and smaller than cobbles (64 to 256 millimetres diameter).

Input: Part of golf is trying to get a higher point total than others.
Knowledge:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

In golf, the objective is to have the lowest score possible, not the highest. Players aim to complete each hole in the fewest number of strokes, with the overall goal being to have the lowest total score at the end of the round.

### 3 LangChain

This is adopted from [this notebook](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/notebooks/pe-chatgpt-langchain.ipynb).

In [57]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [58]:
# chat mode instance
chat = ChatOpenAI(temperature=0)

In [66]:
#Seems to return positive consistently, as expected
USER_INPUT = "I love programming."
FINAL_PROMPT = """Classify the text into neutral, negative or positive.

Text: {user_input}.
Sentiment:"""

chat.invoke([HumanMessage(content=FINAL_PROMPT.format(user_input=USER_INPUT))])

AIMessage(content='Positive', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 27, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d04574b6-39ef-44db-9873-b551ddfb131f-0', usage_metadata={'input_tokens': 27, 'output_tokens': 1, 'total_tokens': 28})

---